# JUPYTER NOTEBOOKS in APT 29 Detection Hackathon!!

### 1. Importing Python Libraries

In [1]:
from pyspark.sql import SparkSession

### 2. Initializing SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.caseSensitive", "true")

### 3. Importing Dataset

In [3]:
!unzip ../datasets/day1/apt29_evals_day1_manual.zip 

Archive:  apt29_evals_day1_manual.zip
  inflating: apt29_evals_day1_manual_2020-05-01225525.json  


In [4]:
df = spark.read.json('apt29_evals_day1_manual_2020-05-01225525.json')

### 4. Initial Exploratory Analysis of Dataset

* What is the **dataframe** structure?

In [5]:
print('It has ',df.count(), ' records')
print('It has ',len(df.columns),' fields')

It has  196081  records
It has  388  fields


* What **Sources** of data do we have?

In [6]:
sources = df.groupBy('Channel').count().orderBy('count',ascending = False).show(truncate = False)

+----------------------------------------------------------------------+------+
|Channel                                                               |count |
+----------------------------------------------------------------------+------+
|Microsoft-Windows-Sysmon/Operational                                  |143884|
|Security                                                              |28627 |
|security                                                              |12375 |
|Microsoft-Windows-PowerShell/Operational                              |5694  |
|Windows PowerShell                                                    |5285  |
|System                                                                |91    |
|Microsoft-Windows-WMI-Activity/Operational                            |90    |
|Microsoft-Windows-TerminalServices-RemoteConnectionManager/Operational|15    |
|Microsoft-Windows-Windows Firewall With Advanced Security/Firewall    |10    |
|Microsoft-Windows-TerminalServices-Loca

* What are the **TOP 10 Security** event logs?

In [7]:
securityEventLogs = df.filter("Channel == 'Security' OR Channel == 'security'").groupBy('EventID').count().orderBy('count',ascending = False).show(10,truncate = False)

+-------+-----+
|EventID|count|
+-------+-----+
|4658   |10973|
|4656   |5497 |
|4690   |5471 |
|4663   |5337 |
|5156   |3163 |
|5447   |2587 |
|5158   |1896 |
|4703   |1802 |
|4673   |934  |
|4688   |460  |
+-------+-----+
only showing top 10 rows



* What **Sysmon** event logs do we have?

In [8]:
sysmonEventLogs = df.filter(df.Channel == 'Microsoft-Windows-Sysmon/Operational').groupBy('EventID').count().orderBy('count',ascending = False).show(truncate = False)

+-------+-----+
|EventID|count|
+-------+-----+
|12     |61151|
|10     |39283|
|7      |20259|
|13     |17541|
|11     |1649 |
|3      |1229 |
|9      |652  |
|1      |447  |
|23     |422  |
|5      |401  |
|18     |362  |
|2      |209  |
|8      |95   |
|17     |84   |
|22     |81   |
|15     |18   |
|4      |1    |
+-------+-----+



* What are the **TOP 10 Processes** that have been created based on **Sysmon Event 1: Process Creation**?

In [9]:
sysmonEid1 = df.filter("Channel == 'Microsoft-Windows-Sysmon/Operational' AND EventID == 1").groupBy('Image').count().orderBy('count',ascending = False).show(10,truncate = False)

+---------------------------------------------------------+-----+
|Image                                                    |count|
+---------------------------------------------------------+-----+
|C:\Windows\System32\svchost.exe                          |100  |
|C:\Windows\System32\conhost.exe                          |65   |
|C:\Windows\System32\backgroundTaskHost.exe               |18   |
|C:\Windows\System32\wbem\WmiPrvSE.exe                    |17   |
|C:\Windows\System32\RuntimeBroker.exe                    |13   |
|C:\Windows\System32\taskhostw.exe                        |13   |
|C:\Windows\System32\cmd.exe                              |12   |
|C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe|12   |
|C:\Windows\System32\rundll32.exe                         |7    |
|C:\Windows\System32\logman.exe                           |7    |
+---------------------------------------------------------+-----+
only showing top 10 rows



* What are the **TOP 10 Pairs of IPs** that have the most connections based on **Sysmon Event 3: Network Connection**?

In [10]:
pairIps = df.filter("Channel == 'Microsoft-Windows-Sysmon/Operational' AND EventID == 3").groupBy(['SourceIp','DestinationIp']).count().orderBy('count',ascending = False).show(10,truncate = False)

+------------------------------+------------------------------+-----+
|SourceIp                      |DestinationIp                 |count|
+------------------------------+------------------------------+-----+
|10.0.1.6                      |192.168.0.4                   |348  |
|10.0.1.4                      |10.0.0.4                      |160  |
|fe80:0:0:0:e40c:95b6:b0a7:6429|fe80:0:0:0:e40c:95b6:b0a7:6429|116  |
|0:0:0:0:0:0:0:1               |0:0:0:0:0:0:0:1               |111  |
|10.0.1.6                      |10.0.0.4                      |88   |
|10.0.0.4                      |10.0.1.4                      |70   |
|10.0.0.4                      |10.0.0.4                      |39   |
|10.0.1.4                      |10.0.1.6                      |34   |
|10.0.0.4                      |172.18.39.2                   |30   |
|10.0.0.4                      |10.0.1.6                      |25   |
+------------------------------+------------------------------+-----+
only showing top 10 

* What are the **TOP 10 Pairs of Ports** that have the most connections based on **Sysmon Event 3: Network Connection**?

In [11]:
pairPorts = df.filter("Channel == 'Microsoft-Windows-Sysmon/Operational' AND EventID == 3").groupBy(['SourcePort','DestinationPort']).count().orderBy('count',ascending = False).show(10,truncate = False)

+----------+---------------+-----+
|SourcePort|DestinationPort|count|
+----------+---------------+-----+
|138       |138            |16   |
|123       |123            |11   |
|137       |137            |8    |
|59997     |445            |4    |
|57275     |135            |2    |
|49739     |135            |2    |
|57146     |389            |2    |
|60132     |445            |2    |
|57269     |9389           |2    |
|60219     |49667          |2    |
+----------+---------------+-----+
only showing top 10 rows



* What examples of **Registry Values** do we have based on **Sysmon Event 13: Registry Event (Value Set)**?

In [12]:
registryValues = df.filter("Channel == 'Microsoft-Windows-Sysmon/Operational' AND EventID == 13").groupBy('TargetObject').count().orderBy('count',ascending = False).show(truncate = 70)

+----------------------------------------------------------------------+-----+
|                                                          TargetObject|count|
+----------------------------------------------------------------------+-----+
|HKLM\System\CurrentControlSet\Services\W32Time\Config\LastKnownGood...|  465|
|  HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\VFUProvider\StartTime|   98|
|HKLM\SOFTWARE\Microsoft\Windows NT\CurrentVersion\Notifications\Dat...|   67|
|HKU\S-1-5-21-1830255721-3727074217-2423397540-1107\Software\Microso...|   62|
|HKU\S-1-5-21-1830255721-3727074217-2423397540-1107\Software\Microso...|   62|
|HKU\S-1-5-20\Software\Microsoft\Windows\CurrentVersion\DeliveryOpti...|   51|
|HKU\S-1-5-20\Software\Microsoft\Windows\CurrentVersion\DeliveryOpti...|   51|
|HKU\S-1-5-21-1830255721-3727074217-2423397540-1107\Software\Microso...|   49|
|HKLM\System\CurrentControlSet\Services\W32Time\SecureTimeLimits\Run...|   42|
|HKLM\System\CurrentControlSet\Services\W32Time\Secu